In [9]:
import os
import sys
import numpy as np
import pandas as pd
import openmatrix as omx

In [10]:
# Assume you have not changed the relative paths after cloning the repository
root = os.path.dirname(os.path.abspath('.'))
root

'D:\\src\\TransportationNetworks'

In [11]:
# We list all folders available, most of which are TNTP instances
folders = [x for x in os.listdir(root)[1:] if os.path.isdir(os.path.join(root, x))]

In [16]:
# Function to import OMX matrices
def import_matrix(matfile):
    f = open(matfile, 'r')
    all_rows = f.read()
    blocks = all_rows.split('Origin')[1:]
    matrix = {}
    for k in range(len(blocks)):
        orig = blocks[k].split('\n')
        dests = orig[1:]
        orig=int(orig[0])

        d = [eval('{'+a.replace(';',',').replace(' ','') +'}') for a in dests]
        destinations = {}
        for i in d:
            destinations = {**destinations, **i}
        matrix[orig] = destinations
    zones = max(matrix.keys())
    mat = np.zeros((zones, zones))
    for i in range(zones):
        for j in range(zones):
            # We map values to a index i-1, as Numpy is base 0
            mat[i, j] = matrix.get(i+1,{}).get(j+1,0)

    index = np.arange(zones) + 1

    myfile = omx.open_file('demand.omx','w')
    myfile['matrix'] = mat
    myfile.create_mapping('taz', index)
    myfile.close()
    return index

In [13]:
# If you want to create AequilibraE graph and matrix
import sys
sys.path.append('D:/src/aequilibrae')
from aequilibrae.paths import Graph

In [62]:
def import_networks(netfile, index):
    network = pd.read_csv(netfile, skiprows=8, sep='\t')
    trimmed= [s.strip().lower() for s in network.columns]
    network.columns = trimmed
    network.drop(['~',';'], axis=1, inplace=True)
    
    aeq_cols = []
    for i in list(network.columns):
        if i == 'init_node':
            aeq_cols.append('a_node')
        elif i == 'term_node':
            aeq_cols.append('b_node')
        elif i == 'b':
            aeq_cols.append('alpha')
        elif i == 'power':
            aeq_cols.append('beta')
        else:
            aeq_cols.append(i)
    network.columns = aeq_cols
    network.fillna(0, inplace=True)
    network = network.assign(direction=1)
    network = network.assign(link_id=np.arange(1,network.shape[0]+1))
    
    g = Graph()
    g.network = network.to_records(index=False)
    g.network_ok = True
    g.status = 'OK'
    g.prepare_graph(index)
    g.set_graph('free_flow_time')
    g.graph
    g.save_to_disk('graph.aeg')

In [105]:
for f in folders:
    mod = os.path.join(root, f)
    os.chdir(mod)
    mod_files = os.listdir('.')
    
    for i in mod_files:
        if 'TRIPS' in i.upper() and i.lower()[-5:]=='.tntp':
            print(f.upper())
            print('    trips')
            index = import_matrix(i)
            print('            {} zones'.format(index.shape[0]))
    for netfile in mod_files:
        if '_NET.' in netfile.upper() and netfile.lower()[-5:]=='.tntp':
            print('    networks')
            import_networks(netfile, index)

ANAHEIM
    trips
            38 zones
    networks
AUSTIN
    trips
            7388 zones
    networks
BARCELONA
    trips
            110 zones
    networks
BERLIN-CENTER
    trips
            865 zones
    networks
BERLIN-FRIEDRICHSHAIN
    trips
            23 zones
    networks
BERLIN-MITTE-CENTER
    trips
            36 zones
    networks
BERLIN-MITTE-PRENZLAUERBERG-FRIEDRICHSHAIN-CENTER
    trips
            98 zones
    networks
BERLIN-PRENZLAUERBERG-CENTER
    trips
            38 zones
    networks
BERLIN-TIERGARTEN
    trips
            26 zones
    networks
BIRMINGHAM-ENGLAND
    trips
            898 zones
    networks
BRAESS-EXAMPLE
    trips
            1 zones
    networks
CHICAGO-SKETCH
    trips
            387 zones
    networks
EASTERN-MASSACHUSETTS
    trips
            74 zones
    networks
GOLDCOAST
    trips
            1068 zones
HESSEN-ASYMMETRIC
    trips
            245 zones
    networks
PHILADELPHIA
    trips
            1525 zones
    networks
SIOUXFALL